In [33]:
import pandas as pd
import pulp
from typing import List, Tuple, Dict


In [34]:

import pandas as pd
import pulp
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Tuple, Dict

plt.style.use("ggplot")

df = pd.read_csv('Nuclear waste management.csv', index_col=0)

preorder = {
    1: [5],
    2: [1],
    3: [3, 7],
    4: [11],
    5: [14],
    6: [22, 23],
    7: [24],
    8: [27]
}

preferential_info = [(5,1),
                     (1,3),
                     (3,11),
                     (11,14),
                     (14,22),
                     (22,24),
                     (24,27)]

indifference_information = [(3, 7),
                            (22, 23)]

In [35]:

criteria = df.columns.tolist()
alternatives = {x for t in (preferential_info + indifference_information) for x in t}

pulp.LpSolverDefault.msg = 0
problem = pulp.LpProblem("UTA", pulp.LpMaximize)
print("Kryteria:", criteria)

# VARIABLES
u_vars = {}
for alternative in alternatives:
    for criterion in criteria:
        value = df.loc[alternative, criterion]

    
        u_vars[(criterion, value)] = pulp.LpVariable(
            f"u{criterion}_{value}", lowBound=0
        )

print(u_vars)



Kryteria: ['C1', 'C2', 'C3', 'C4']
{('C1', 0.6): uC1_0.6, ('C2', 0.93): uC2_0.93, ('C3', 0.0): uC3_0.0, ('C4', 0.73): uC4_0.73, ('C1', 1.0): uC1_1.0, ('C2', 0.45): uC2_0.45, ('C3', 0.57): uC3_0.57, ('C4', 0.5): uC4_0.5, ('C1', 0.62): uC1_0.62, ('C2', 0.4): uC2_0.4, ('C3', 0.56): uC3_0.56, ('C1', 0.4): uC1_0.4, ('C2', 0.9): uC2_0.9, ('C4', 0.82): uC4_0.82, ('C1', 0.61): uC1_0.61, ('C2', 0.54): uC2_0.54, ('C3', 0.38): uC3_0.38, ('C4', 0.49): uC4_0.49, ('C1', 0.69): uC1_0.69, ('C2', 0.49): uC2_0.49, ('C4', 0.61): uC4_0.61, ('C1', 0.32): uC1_0.32, ('C2', 0.83): uC2_0.83, ('C4', 0.94): uC4_0.94, ('C1', 0.59): uC1_0.59, ('C2', 0.24): uC2_0.24, ('C3', 0.7): uC3_0.7, ('C4', 0.63): uC4_0.63, ('C1', 0.73): uC1_0.73, ('C2', 0.03): uC2_0.03, ('C3', 1.0): uC3_1.0, ('C1', 0.8): uC1_0.8, ('C2', 0.06): uC2_0.06, ('C4', 0.67): uC4_0.67}


In [36]:
epsilon = pulp.LpVariable("epsilon", lowBound=0, upBound=1)
problem += epsilon

# PRE-ORDER
for a, b in preferential_info:
    problem += (
        pulp.lpSum(u_vars[(c, df.loc[a, c])] for c in criteria)
        >= pulp.lpSum(u_vars[(c, df.loc[b, c])] for c in criteria) + epsilon
    )

for a, b in indifference_information:
    problem += pulp.lpSum(u_vars[(c, df.loc[a, c])] for c in criteria) == pulp.lpSum(
        u_vars[(c, df.loc[b, c])] for c in criteria
    )

In [37]:
worst_values = {criterion: df[criterion].max() for criterion in criteria}
best_values = {criterion: df[criterion].min() for criterion in criteria}


u_best = []
u_worst = []

for criterion, value in worst_values.items():
    if (criterion, value) not in u_vars:
        u_vars[(criterion, value)] = pulp.LpVariable(f"u{criterion}_{value}", lowBound=0)

    u_worst.append(u_vars[(criterion, value)])

for criterion, value in best_values.items():
    if (criterion, value) not in u_vars:
        u_vars[(criterion, value)] = pulp.LpVariable(f"u{criterion}_{value}", lowBound=0)
        
    u_best.append(u_vars[(criterion, value)])

print("U worst:", u_worst)
print("U best:", u_best)

problem += pulp.lpSum(u_worst) == 0
problem += pulp.lpSum(u_best) == 1

for criterion in criteria:
    problem += u_vars[(criterion, best_values[criterion])] >= u_vars[(criterion, worst_values[criterion])]

U worst: [uC1_1.0, uC2_1.0, uC3_1.0, uC4_1.0]
U best: [uC1_0.32, uC2_0.03, uC3_0.0, uC4_0.49]


In [38]:

#print(problem)
problem.solve()
print("Status:", pulp.LpStatus[problem.status])

for v in problem.variables():
    print(v.name, "=", v.varValue)

Status: Optimal
epsilon = 1.0
uC1_0.32 = 1.0
uC1_0.4 = 5.0
uC1_0.59 = 2.0
uC1_0.6 = 6.0
uC1_0.61 = 4.0
uC1_0.62 = 7.0
uC1_0.69 = 3.0
uC1_0.73 = 1.0
uC1_0.8 = 0.0
uC1_1.0 = 0.0
uC2_0.03 = 0.0
uC2_0.06 = 0.0
uC2_0.24 = 0.0
uC2_0.4 = 0.0
uC2_0.45 = 0.0
uC2_0.49 = 0.0
uC2_0.54 = 0.0
uC2_0.83 = 0.0
uC2_0.9 = 0.0
uC2_0.93 = 0.0
uC2_1.0 = 0.0
uC3_0.0 = 0.0
uC3_0.38 = 0.0
uC3_0.56 = 0.0
uC3_0.57 = 5.0
uC3_0.7 = 0.0
uC3_1.0 = 0.0
uC4_0.49 = 0.0
uC4_0.5 = 0.0
uC4_0.61 = 0.0
uC4_0.63 = 0.0
uC4_0.67 = 0.0
uC4_0.73 = 0.0
uC4_0.82 = 0.0
uC4_0.94 = 1.0
uC4_1.0 = 0.0
